# Bat Hunting Calls

In [13]:
import io
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter

Konstanten:

In [2]:
db_file_name = "batcallsv14.db"
table_name = "batcalls"

SQL-Datenbank einlesen:

In [9]:
data = pd.read_sql(table_name, f"sqlite:///{db_file_name}")
data.groupby('bat').agg('count')

Index(['call', 'target', 'id', 'bat', 'arr', 'db'], dtype='object')
                       call  target    id   arr    db
bat                                                  
Bartfledermaus         2393    2393  2393  2393  2393
Bechsteinfledermaus     329     329   329   329   329
Breitflügelfledermaus  2681    2681  2681  2681  2681
Fransenfledermaus      3040    3040  3040  3040  3040
GroßerAbendsegler      3005    3005  3005  3005  3005
GroßesMausohr          2366    2366  2366  2366  2366
KleinerAbendsegler     2884    2884  2884  2884  2884
Langohr                 411     411   411   411   411
Mopsfledermaus          327     327   327   327   327
Mückenfledermaus        566     566   566   566   566
Nordfledermaus         1245    1245  1245  1245  1245
Rauhautfledermaus      4695    4695  4695  4695  4695
Rauschen               4713    4713  4713  4713  4713
Teichfledermaus        2142    2142  2142  2142  2142
Wasserfledermaus       6074    6074  6074  6074  6074
Weißrandfleder

Serialisierte Numpy-Arrays einlesen:

In [10]:
data['arr'] = data['arr'].map(lambda bytes: np.load(io.BytesIO(bytes)))
data

,call,target,id,bat,arr,db
0,38818,0,0,Bartfledermaus,"[157, 109, -92, -126, -10, 40, 11, -41, 45, 21...",skiba
1,46731,0,1,Bartfledermaus,"[-487, -92, 334, 561, 530, 229, -180, -511, -5...",skiba
2,49137,0,2,Bartfledermaus,"[-5, 120, 23, -22, 47, 15, -117, -154, -146, -...",skiba
3,49557,0,3,Bartfledermaus,"[-189, -179, -79, 34, 95, 37, -7, 35, 30, 26, ...",skiba
4,22197,0,4,Bartfledermaus,"[1, 0, -1, 0, 2, 5, 6, 7, 6, 5, 1, -4, -6, -9,...",skiba
...,...,...,...,...,...,...
51714,11775,18,7880,Zwergfledermaus,"[3, 91, 32, -82, -77, -37, -20, -2, 66, 30, -6...",skiba
51715,38802,18,7881,Zwergfledermaus,"[-14, 64, 27, -17, 22, 12, -8, 16, 13, 9, 10, ...",skiba
51716,25322,18,7882,Zwergfledermaus,"[-27, 92, 54, -27, 14, 10, -16, 13, 64, 33, -5...",skiba
51717,23866,18,7883,Zwergfledermaus,"[61, 80, 86, -33, -87, 29, 11, -26, 40, 70, 90...",skiba


Umwandlung int16 -> float32, Normalisierung auf [-1;1]

In [11]:
data['arr'] = data['arr'].map(lambda call: call.astype(np.float32) / 2.0**(16-1))
data

,call,target,id,bat,arr,db
0,38818,0,0,Bartfledermaus,"[0.0047912598, 0.003326416, -0.0028076172, -0....",skiba
1,46731,0,1,Bartfledermaus,"[-0.014862061, -0.0028076172, 0.010192871, 0.0...",skiba
2,49137,0,2,Bartfledermaus,"[-0.00015258789, 0.0036621094, 0.0007019043, -...",skiba
3,49557,0,3,Bartfledermaus,"[-0.0057678223, -0.0054626465, -0.0024108887, ...",skiba
4,22197,0,4,Bartfledermaus,"[3.0517578e-05, 0.0, -3.0517578e-05, 0.0, 6.10...",skiba
...,...,...,...,...,...,...
51714,11775,18,7880,Zwergfledermaus,"[9.1552734e-05, 0.0027770996, 0.0009765625, -0...",skiba
51715,38802,18,7881,Zwergfledermaus,"[-0.0004272461, 0.001953125, 0.0008239746, -0....",skiba
51716,25322,18,7882,Zwergfledermaus,"[-0.0008239746, 0.0028076172, 0.0016479492, -0...",skiba
51717,23866,18,7883,Zwergfledermaus,"[0.0018615723, 0.0024414062, 0.0026245117, -0....",skiba


Bandpass-Filter, nur Frequenzen zwischen 1500Hz und 12,000Hz

In [14]:
def butter_bandpass(lowcut, highcut, fs, order=5):
  nyq = 0.5 * fs
  low = lowcut / nyq
  high = highcut / nyq
  b, a = butter(order, [low, high], btype='band')
  return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
  b, a = butter_bandpass(lowcut, highcut, fs, order=order)
  y = lfilter(b, a, data)
  return y

data['arr'] = data['arr'].map(lambda call: butter_bandpass_filter(call, 1500, 12000, 44100, 5))
data

,call,target,id,bat,arr,db
0,38818,0,0,Bartfledermaus,"[0.00020882230893271082, 0.0009980651541837105...",skiba
1,46731,0,1,Bartfledermaus,"[-0.0006477481812116571, -0.002768565801001311...",skiba
2,49137,0,2,Bartfledermaus,"[-6.6503920042264594e-06, 0.000132441044773677...",skiba
3,49557,0,3,Bartfledermaus,"[-0.00025138481775976014, -0.00126504816754052...",skiba
4,22197,0,4,Bartfledermaus,"[1.3300784008452918e-06, 5.433672665551425e-06...",skiba
...,...,...,...,...,...,...
51714,11775,18,7880,Zwergfledermaus,"[3.990235202535875e-06, 0.00013733815247357584...",skiba
51715,38802,18,7881,Zwergfledermaus,"[-1.8621097611834085e-05, 9.053600336378722e-0...",skiba
51716,25322,18,7882,Zwergfledermaus,"[-3.591211682282288e-05, -2.4341949092121615e-...",skiba
51717,23866,18,7883,Zwergfledermaus,"[8.11347824515628e-05, 0.00043786030466626025,...",skiba


Normalisierung:

In [15]:
data['arr'] = data['arr'].map(lambda call: (call - np.mean(call)) / np.std(call))
data

,call,target,id,bat,arr,db
0,38818,0,0,Bartfledermaus,"[0.023500146989869186, 0.11215443193498724, 0....",skiba
1,46731,0,1,Bartfledermaus,"[-0.08146524693426499, -0.3480960720001129, -0...",skiba
2,49137,0,2,Bartfledermaus,"[-0.0017242419015283993, 0.03306417655714896, ...",skiba
3,49557,0,3,Bartfledermaus,"[-0.03105441485428307, -0.1564046011958408, -0...",skiba
4,22197,0,4,Bartfledermaus,"[-2.5258193459459957e-05, 9.13013452862939e-06...",skiba
...,...,...,...,...,...,...
51714,11775,18,7880,Zwergfledermaus,"[7.319888459029453e-05, 0.002010176507665997, ...",skiba
51715,38802,18,7881,Zwergfledermaus,"[-0.001567426754048205, 0.000620758417820513, ...",skiba
51716,25322,18,7882,Zwergfledermaus,"[-0.0006014420564289893, -0.000404535760215012...",skiba
51717,23866,18,7883,Zwergfledermaus,"[0.0008649904359816796, 0.004397458837696364, ...",skiba
